In [1]:
%load_ext autoreload
%autoreload 2
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, diff_of_days, diff_of_times
from glob import glob
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
import datetime
import math
import os
import numpy as np
import pandas as pd
import sys

HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/github/data_mining/library")
from parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
               or path.count(COLUMN_ID) 
               or path.count('Fraud') 
               or path.count('D1')
               or path.count('D3')
               or path.count('C1')
               or path.count('C13')
               or path.count('V221')
               or path.count('V285')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
               or path.count('M')
               or path.count('Product')
              ]
test_paths  = [path for path in test_paths  
               if path.count('DT') 
               or path.count(COLUMN_ID) 
               or path.count('Fraud') 
               or path.count('D1')
               or path.count('D3')
               or path.count('C1')
               or path.count('C13')
               or path.count('V221')
               or path.count('V285')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
               or path.count('M')
               or path.count('Product')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)

In [3]:
# START_DATE = '2017-11-04'
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
df_train['datetime'] = df_train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
df_test['datetime'] = df_test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
df_train['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
df_test['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
df_train['date'] = df_train['datetime'].map(lambda x: x.date())
df_test['date']  =  df_test['datetime'].map(lambda x: x.date())
df_train['hour'] = df_train['datetime'].map(lambda x: x.hour)
df_test['hour']  =  df_test['datetime'].map(lambda x: x.hour)

In [4]:
def trans_DT(df):
    list_regist = []
    for d, diff in tqdm(df[['date', 'D1']].values):
        if diff < 999999:
            regist = date_add_days(d, -1*diff)
        else:
            regist = date_add_days(d, 0)
        list_regist.append(str(regist))
    df['Regist_date'] = list_regist
    return df
df_train = trans_DT(df_train)
df_test  = trans_DT(df_test)

100%|██████████| 506691/506691 [00:01<00:00, 265827.79it/s]


In [5]:
df_train['is_train'] = 1
df_test['is_train']  = 0
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)

異なるuser keyで様々なユーザー特定パターン（正確でなくてもよい）を作り、それぞれのパターンで特徴を作る  
正確でなくてもよい理由としては、粒度を粗くしてもある条件を外した特徴量になるだけなので、問題ない

In [6]:
user_info = [
    'ProductCD',
#     'Regist_date'
]
list_card = [
    'card1',
    'card2',
    'card3',
    'card4',
#     'card5',
#     'card6',
]
list_addr = [
    'addr1',
    'addr2',
]
list_pemail = [
    'P_emaildomain'
]
list_M = [col for col in data.columns if col.startswith('M')]

user_keys__card = user_info + list_card

user_keys__card_addr = user_info + list_card + list_addr
user_keys__card_pemail = user_info + list_card + list_pemail
user_keys__card_M = user_info + list_card + list_M

user_keys__card_addr_pemail = user_info + list_card + list_addr + list_pemail
user_keys__card_addr_M = user_info + list_card + list_addr + list_M
user_keys__card_pemail_M = user_info + list_card + list_pemail + list_M

user_keys__card_addr_pemail_M = user_info + list_card + list_addr + list_pemail + list_M

list_user_keys = [
  ['user_keys__card',      user_keys__card,],
  ['user_keys__card_addr',      user_keys__card_addr,],
  ['user_keys__card_pemail',      user_keys__card_pemail,],
  ['user_keys__card_M',      user_keys__card_M,],
  ['user_keys__card_addr_M',      user_keys__card_addr_M,],
  ['user_keys__card_addr_pemail',      user_keys__card_addr_pemail,],
  ['user_keys__card_pemail_M',      user_keys__card_pemail_M,],
  ['user_keys__card_addr_pemail_M',      user_keys__card_addr_pemail_M,],
]
check_Ds = ['D1', 'D2', 'D3', 'D4', 'D5', 'datetime']

data['D1'].fillna(-1, inplace=True)
data[user_info + list_card] = data[user_info + list_card].astype('str').fillna('#')

# user_keys = user_keys__card_addr
user_keys = user_keys__card_addr_pemail
# user_keys = user_keys__card_addr_pemail_M

uid = 'user_id'
df = data.copy()

for col in user_keys:
    df[col].fillna('#', inplace=True)

df[user_keys] =  df[user_keys].astype('str')

In [7]:
print(user_keys)

['ProductCD', 'card1', 'card2', 'card3', 'card4', 'addr1', 'addr2', 'P_emaildomain']


user_keyで粗いグループを作ったら、そのグループの中で同じユーザーを探索し、条件が合えば仲間にしてさらにグループを作っていくイメージ

In [8]:
user_df = df.groupby(user_keys)[COLUMN_ID].min()
user_df = user_df.to_frame(uid)
user_df = df.set_index(user_keys).join(user_df)

check_cols = [COLUMN_ID, uid, COLUMN_TARGET, 'datetime', 'Regist_date', 'is_train'] + ['D1', 'D3', 'C1', 'C13', 'V221', 'V285']
user_df = user_df.reset_index()[check_cols]

user_df['C1']  += 1
user_df['C13'] += 1

if uid not in user_df.columns:
    user_df.reset_index(inplace=True)
    
user_df['hour'] = user_df['datetime'].map(lambda x: x.hour)
user_df['D3'].fillna(-99999, inplace=True)
user_df['C1'].fillna(-99999, inplace=True)
user_df['C13'].fillna(-99999, inplace=True)
user_df['V221'].fillna(-99999, inplace=True)
user_df['V285'].fillna(-99999, inplace=True)

train_max_datetime = df_train['datetime'].max()
test_min_datetime = df_train['datetime'].min()

# user_df.sort_values(by=[uid, 'datetime'], inplace=True)

# user_df['diff_from_regist'] = user_df['datetime'] - user_df['Regist_date'].map(lambda x: datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10]) ) )
# user_df['diff_from_regist'] = user_df['diff_from_regist'].map(lambda x: x.days)

# user_df['Regist_date_add_D1'] = user_df[['Regist_date', 'D1']].apply(lambda x: date_add_days(x[0], x[1]), axis=1)
# user_df['Regist_date_add_D1-D3'] = user_df[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]), axis=1)
# user_df['Regist_date_add_D1-D3-1'] = user_df[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]-1), axis=1)

# user_df['date__before1'] = user_df.groupby(uid)['date'].shift(1)
# user_df['date__before2'] = user_df.groupby(uid)['date'].shift(2)
# user_df['date__before3'] = user_df.groupby(uid)['date'].shift(3)
# user_df['D1__before1']   = user_df.groupby(uid)['D1'].shift(1)
# user_df['D1__before2']   = user_df.groupby(uid)['D1'].shift(2)

# user_df['diff_D1__before1'] = user_df[['D1', 'D1__before1']].apply(lambda x: x[0] - x[1] , axis=1)
# user_df['diff_D1__before2'] = user_df[['D1', 'D1__before2']].apply(lambda x: x[0] - x[1] , axis=1)
# user_df['diff_day__before1'] = user_df[['date', 'date__before1']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
# user_df['diff_day__before2'] = user_df[['date', 'date__before2']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
# user_df['diff_day__before3'] = user_df[['date', 'date__before3']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)

# user_df.to_csv('../output/0901_ieee__user_df_card_addr_pemail__for_specific_same_user.csv', index=False)

In [9]:
def parallel_explore_match_user(df_base, group_no_list):
    
    list_user_map = []
    for group_no in tqdm(group_no_list):

        group = df_base[df_base[uid]==group_no]
        group.sort_values(by='datetime', inplace=True)
        users = group[col_no].unique().tolist()
        user_map = {}
        cnt = 0
        
        while True:
        
            remain_user = group[~group[col_no].isin(list(user_map.keys()))][col_no].values
            if len(remain_user)==0:
                break
        #     remain_user = list( set(users) - set(list(user_map.keys())) )
        
            base_user = remain_user[0]
            user_map[base_user] = base_user
        
            base = group[group[col_no]==base_user]
            base_reg = base['Regist_date'].values[0]
            base_dt = base['datetime'].values[0]
            base_d1 = base['D1'].values[0]
            base_d3 = base['D3'].values[0]
            base_c1 = base['C1'].values[0]
            base_c13 = base['C13'].values[0]
            base_v221 = base['V221'].values[0]
            base_v285 = base['V285'].values[0]
            base_hour = base['hour'].values[0]
            base_fraud = base[COLUMN_TARGET].values[0]
            base_is_train = base['is_train'].values[0]
        
            for i in range(1, len(remain_user), 1):
        
                tmp_user = remain_user[i]
        
                tmp = group[group[col_no]==tmp_user]
                tmp_reg = tmp['Regist_date'].values[0]
                diff_regist_days = abs((pd.to_datetime(tmp_reg) - pd.to_datetime(base_reg)).days)
                
                # Regist_dateで1day以上の差は除外
                if diff_regist_days>1:
                    continue
                
                tmp_dt = tmp['datetime'].values[0]
                tmp_d1 = tmp['D1'].values[0]
                tmp_d3 = tmp['D3'].values[0]
                tmp_c1 = tmp['C1'].values[0]
                tmp_c13 = tmp['C13'].values[0]
                tmp_v221 = tmp['V221'].values[0]
                tmp_v285 = tmp['V285'].values[0]
                tmp_hour = tmp['hour'].values[0]
                tmp_fraud = tmp[COLUMN_TARGET].values[0]
                tmp_is_train = tmp['is_train'].values[0]
                
                is_same_data = base_is_train == tmp_is_train
                
                # Train, Testの境の場合
                if not is_same_data:
                    if tmp_d3==-99999:
                        match_d1 = False
                    else:
                        no_term_date = date_add_days(pd.to_datetime(base_dt), tmp_d3*-1)
                        if train_max_datetime <= no_term_date <= test_min_datetime:
                            match_d1 = True
                        else:
                            match_d1 = False
                else:
                    match_d1 = False
        
                diff_d1   = tmp_d1 - base_d1
                diff_c1   = abs(tmp_c1 - base_c1)
                minus_c1   = tmp_c1 < base_c1
                if minus_c1:
                    thres_c1 = 1
                    thres_c1_2 = 1
                else:
                    thres_c1 = 1
                    thres_c1_2 = 1
                thres_c1_large = 3
                thres_c1_ratio_low = 0.8
                thres_c1_ratio_high = 1.2
                thres_v = 1
                
        
                diff_c13  = abs(tmp_c13 - base_c13)
                diff_v221 = abs(tmp_v221 - base_v221)
                diff_v285 = abs(tmp_v285 - base_v285)
        
                ratio_c1   = abs(tmp_c1  / base_c1)
                ratio_c13  = abs(tmp_c13 / base_c13)
        
        #         print("="*12)
        #         print(f"base {base_user} {base_dt}  ->  D1:{base_d1} D3:{base_d3} C1:{base_c1} C13:{base_c13} V221:{base_v221} V285:{base_v285}")
        #         print(f"tmp  {tmp_user} {tmp_dt}  ->  D1:{tmp_d1} D3:{tmp_d3} C1:{tmp_c1} C13:{tmp_c13} V221:{tmp_v221} V285:{tmp_v285}")
        #         print(f"diff_D1 {diff_d1} diffD1-D3: {diff_d1-tmp_d3} diff_C1:{diff_c1} diff_C13:{diff_c13} diff_V221:{diff_v221} diff_V285:{diff_v285}")
        #         print("="*12)
        
                # D3がNullの場合
                if base_d3==-99999 or tmp_d3==-99999:
                    # TODO
                    # ここでFraud条件を入れると、train-test間を同じユーザーにまとめることはできなくなる
                    # 両パターンを作成し、Fraud含めないパターンでまたいたユーザーが本当にまたいでるかチェックしたい
                    
                    if diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (thres_c1_ratio_low<=ratio_c1 and ratio_c1<=thres_c1_ratio_high) and (0.4<=ratio_c13 or ratio_c13<=thres_c1_ratio_high) and diff_v221<=thres_v and diff_v285<=thres_v:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (thres_c1_ratio_low<=ratio_c1 and ratio_c1<=thres_c1_ratio_high) and (0.4<=ratio_c13 or ratio_c13<=thres_c1_ratio_high) and diff_v221>90000 and diff_v285>90000:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (thres_c1_ratio_low<=ratio_c1 and ratio_c1<=thres_c1_ratio_high) and (0.4<=ratio_c13 or ratio_c13<=thres_c1_ratio_high) and diff_v221>90000 and diff_v285<=thres_v:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (thres_c1_ratio_low<=ratio_c1 and ratio_c1<=thres_c1_ratio_high) and (0.4<=ratio_c13 or ratio_c13<=thres_c1_ratio_high) and diff_v221<=thres_v and diff_v285>90000:
                        pass
                    else:
                        continue
                    
        #             if base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
        #                 pa
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
        #                 pass
        #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
        #                 pass
        #             else:
        #                 continue
        
                # D3がNullsでない場合
                # D3の差分が時差を考慮してもおかしい場合は別ユーザーにする（時差が1~2時間以上だと強制的にはじいてしまうが）
                elif (diff_d1 == tmp_d3) or (abs(diff_d1 - tmp_d3)==1):
                    pass
                else:
                    # train testの境目
                    if not is_same_data and match_d1:
                        pass
                    else:
                        # 別ユーザーとみなす
                        continue
        
                # D3がNullsでない場合
                if base_d3!=-99999 and tmp_d3!=-99999:
                    
                    # 別ユーザーとみなす
                    if diff_c1<-10 or diff_c1>1000:
                        continue
                    if diff_c13<-10 or diff_c13>1000:
                        continue
        
        #             print("  * check diff_V")
                    # V221, 285の変化量が1トランザクションで10以上はないという想定
                    if (10<= diff_v221) and (diff_v221<=100):
                        continue
                    if (10<= diff_v285) and (diff_v285<=100):
                        continue
        
        #             print("  * check diff_C")
                    if diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1>0:
        #                 print('    * pass1')
                        pass
                    elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1==0:
                        pass
                    elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (thres_c1_ratio_low<=ratio_c1 and ratio_c1<=thres_c1_ratio_high) and (thres_c1_ratio_low<=ratio_c13 or ratio_c13<=thres_c1_ratio_high) and diff_d1!=0:
                        pass
                    else:
                        # 別ユーザーとみなす
                        continue
        
                user_map[tmp_user] = base_user
        #         print(pd.Series(user_map))
                
                # base_userと同じと判定されたので、baseを直近のトランザクションに書き換える
                base_dt    = tmp['datetime'].values[0]
                base_d1    = tmp['D1'].values[0]
                base_d3    = tmp['D3'].values[0]
                base_c1    = tmp['C1'].values[0]
                base_c13   = tmp['C13'].values[0]
                base_v221  = tmp['V221'].values[0]
                base_v285  = tmp['V285'].values[0]
                base_hour  = tmp['hour'].values[0]
                base_fraud = tmp[COLUMN_TARGET].values[0]
        
            if len(user_map.keys()) == len(users):
                break
#             break
                
        ser_user = pd.Series(user_map).to_frame('same_user')
        ser_user['group_no'] = group_no
        list_user_map.append(ser_user)
    return list_user_map

In [10]:
col_no = 'no'
user_df['no'] = np.arange(user_df.shape[0])+1

tmp = user_df.groupby(uid)['C1'].count().to_frame('user_cnt')
df_multi_user = tmp[tmp['user_cnt']>1]
multi_user = df_multi_user.index.tolist()
df_multi = user_df[user_df[uid].isin(multi_user)]

arg_list = get_parallel_arg_list(n_jobs=60, arg_list=multi_user)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(232438, 14)
(90686, 14)
(69455, 14)
(44192, 14)
(37520, 14)
(30199, 14)
(26104, 14)
(23528, 14)
(21212, 14)
(21055, 14)
(17844, 14)
(15229, 14)
(14809, 14)
(15319, 14)
(13053, 14)
(12540, 14)
(12295, 14)
(11215, 14)
(10808, 14)
(10350, 14)
(9265, 14)
(9248, 14)
(8212, 14)
(8611, 14)
(10730, 14)
(11139, 14)
(10768, 14)
(10528, 14)
(9733, 14)
(9221, 14)
(8795, 14)
(8773, 14)
(8776, 14)
(8634, 14)
(8201, 14)
(9285, 14)
(8190, 14)
(7479, 14)
(7236, 14)
(6898, 14)
(6753, 14)
(6479, 14)
(6311, 14)
(6382, 14)
(5873, 14)
(5448, 14)
(5227, 14)
(5318, 14)
(6523, 14)
(5648, 14)
(5182, 14)
(5263, 14)
(4779, 14)
(4844, 14)
(4596, 14)
(4554, 14)
(4494, 14)
(4089, 14)
(3559, 14)
(3358, 14)


In [11]:
list_p = Parallel(48)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[12:], arg_list[12:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg1 = pd.concat(list_df, axis=0)

100%|██████████| 48/48 [00:15<00:00,  3.37it/s]


In [12]:
remain_users = []
for args in  arg_list[:12]:
    remain_users += args
arg_list = get_parallel_arg_list(n_jobs=60, arg_list=remain_users)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(66455, 14)
(51443, 14)
(50842, 14)
(35155, 14)
(28337, 14)
(22300, 14)
(21898, 14)
(15637, 14)
(16222, 14)
(14701, 14)
(17217, 14)
(17068, 14)
(13947, 14)
(10426, 14)
(10002, 14)
(9391, 14)
(8304, 14)
(10625, 14)
(8546, 14)
(7936, 14)
(7889, 14)
(7664, 14)
(7260, 14)
(7107, 14)
(7324, 14)
(6335, 14)
(6283, 14)
(6383, 14)
(5937, 14)
(5288, 14)
(5560, 14)
(4933, 14)
(4621, 14)
(6034, 14)
(4777, 14)
(4697, 14)
(4688, 14)
(4060, 14)
(5170, 14)
(5210, 14)
(4395, 14)
(3627, 14)
(4789, 14)
(4784, 14)
(3684, 14)
(4322, 14)
(3785, 14)
(3809, 14)
(4412, 14)
(4825, 14)
(3091, 14)
(3587, 14)
(3578, 14)
(3731, 14)
(3550, 14)
(3072, 14)
(2834, 14)
(3351, 14)
(3402, 14)
(3162, 14)


In [13]:
list_p = Parallel(48)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[12:], arg_list[12:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg2 = pd.concat(list_df, axis=0)

100%|██████████| 48/48 [00:03<00:00, 14.55it/s]


In [14]:
remain_users = []
for args in  arg_list[:12]:
    remain_users += args
arg_list = get_parallel_arg_list(n_jobs=60, arg_list=remain_users)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(11931, 14)
(22360, 14)
(15071, 14)
(6409, 14)
(10669, 14)
(15743, 14)
(8490, 14)
(14020, 14)
(6294, 14)
(6549, 14)
(4885, 14)
(13508, 14)
(10562, 14)
(13803, 14)
(6296, 14)
(8765, 14)
(6387, 14)
(8743, 14)
(5893, 14)
(6207, 14)
(5448, 14)
(5402, 14)
(7374, 14)
(5067, 14)
(5490, 14)
(5617, 14)
(3930, 14)
(3044, 14)
(4697, 14)
(4179, 14)
(7354, 14)
(3583, 14)
(5484, 14)
(2631, 14)
(3352, 14)
(3892, 14)
(2507, 14)
(2244, 14)
(4094, 14)
(3125, 14)
(2897, 14)
(5039, 14)
(4092, 14)
(2088, 14)
(2582, 14)
(2016, 14)
(2345, 14)
(2154, 14)
(2771, 14)
(5036, 14)
(1926, 14)
(2785, 14)
(2877, 14)
(4459, 14)
(4995, 14)
(2875, 14)
(2476, 14)
(4489, 14)
(4018, 14)
(4256, 14)


In [15]:
list_p = Parallel(45)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[15:], arg_list[15:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg3 = pd.concat(list_df, axis=0)

100%|██████████| 45/45 [00:00<00:00, 79.43it/s]


In [16]:
remain_users = []
for args in  arg_list[:15]:
    remain_users += args
arg_list = get_parallel_arg_list(n_jobs=60, arg_list=remain_users)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(3293, 14)
(1606, 14)
(2509, 14)
(4523, 14)
(11646, 14)
(6285, 14)
(2568, 14)
(1861, 14)
(918, 14)
(8688, 14)
(3615, 14)
(1850, 14)
(818, 14)
(2816, 14)
(1587, 14)
(1188, 14)
(789, 14)
(1521, 14)
(4344, 14)
(4015, 14)
(4213, 14)
(1362, 14)
(9336, 14)
(832, 14)
(4384, 14)
(1900, 14)
(790, 14)
(1416, 14)
(2221, 14)
(3979, 14)
(2817, 14)
(5003, 14)
(304, 14)
(3685, 14)
(1502, 14)
(803, 14)
(1417, 14)
(1567, 14)
(2829, 14)
(736, 14)
(1366, 14)
(1460, 14)
(1270, 14)
(789, 14)
(8518, 14)
(1628, 14)
(1096, 14)
(2266, 14)
(2304, 14)
(2730, 14)
(1746, 14)
(3782, 14)
(3050, 14)
(3261, 14)
(5053, 14)
(2439, 14)
(506, 14)
(2101, 14)
(2161, 14)
(1528, 14)


In [17]:
list_p = Parallel(40)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[20:], arg_list[20:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg4 = pd.concat(list_df, axis=0)

100%|██████████| 40/40 [00:00<00:00, 286.62it/s]


In [18]:
remain_users = []
for args in  arg_list[:20]:
    remain_users += args
arg_list = get_parallel_arg_list(n_jobs=60, arg_list=remain_users)
arg_df_list = []
for user_ids in arg_list:
    tmp = user_df[user_df[uid].isin(user_ids)]
    print(tmp.shape)
    arg_df_list.append(tmp)

(165, 14)
(656, 14)
(2470, 14)
(92, 14)
(218, 14)
(785, 14)
(555, 14)
(259, 14)
(2032, 14)
(204, 14)
(3491, 14)
(664, 14)
(340, 14)
(434, 14)
(1971, 14)
(9165, 14)
(768, 14)
(4146, 14)
(1380, 14)
(78, 14)
(19, 14)
(619, 14)
(3134, 14)
(273, 14)
(156, 14)
(217, 14)
(77, 14)
(349, 14)
(485, 14)
(171, 14)
(2058, 14)
(6342, 14)
(349, 14)
(102, 14)
(2288, 14)
(1032, 14)
(755, 14)
(626, 14)
(437, 14)
(270, 14)
(319, 14)
(72, 14)
(268, 14)
(359, 14)
(1965, 14)
(427, 14)
(248, 14)
(576, 14)
(730, 14)
(412, 14)
(339, 14)
(424, 14)
(460, 14)
(178, 14)
(112, 14)
(259, 14)
(426, 14)
(137, 14)
(845, 14)
(8252, 14)


In [19]:
list_p = Parallel(60)([delayed(parallel_explore_match_user)(d, gnl) for d, gnl in zip(arg_df_list[:], arg_list[:])])
list_df = []
for tmp_list in tqdm(list_p):
    tmp = pd.concat(tmp_list, axis=0)
    list_df.append(tmp)
df_arg5 = pd.concat(list_df, axis=0)

100%|██████████| 60/60 [00:00<00:00, 740.76it/s]


In [20]:
df_same = pd.concat([df_arg1, df_arg2, df_arg3, df_arg4, df_arg5], axis=0)
print(df_same.shape)
df_same.head()

(1014253, 2)


,same_user,group_no
248388,248388,3029153
248389,248389,3029153
500906,500906,3029161
500907,500906,3029161
500908,500906,3029161


In [21]:
user_df.set_index('no', inplace=True)
user_df['same_user'] = df_same['same_user']
same_user_ids = user_df['same_user'].unique()
same_map = {}
for i, same_id in tqdm(enumerate(same_user_ids[1:])):
    same_map[same_id] = i
cnt = max(same_map.values())+1
new_id_list = []
for same_id in user_df['same_user'].values:
    if same_id in same_map:
        new_id = same_map[same_id]
    else:
        new_id = cnt
        cnt += 1
    new_id_list.append(new_id)

print(user_df.shape)
user_df['predicted_user_id'] = new_id_list
user_df.rename(columns={uid:'section_id'}, inplace=True)
user_df.sort_values(by='predicted_user_id', inplace=True)
# user_df[[COLUMN_ID, 'predicted_user_id', 'section_id']].to_csv('../output/same_user_pattern/0905__hard_same_user_id__card_addr.csv', index=False)
user_df[[COLUMN_ID, 'predicted_user_id', 'section_id']].to_csv('../output/same_user_pattern/0905__hard_same_user_id__card_addr_pemail.csv', index=False)
# user_df[[COLUMN_ID, 'predicted_user_id', 'section_id']].to_csv('../output/same_user_pattern/0905__hard_same_user_id__card_addr_pemail_M.csv', index=False)

471230it [00:00, 1643751.96it/s]


(1097231, 14)


In [61]:
user_df[[COLUMN_ID, 'predicted_user_id', 'section_id']]

,TransactionID,predicted_user_id,section_id
no,,,
3,3328484,0,3328484
4,3337343,0,3328484
5,3337365,0,3328484
9,3250462,1,3250462
10,3346768,2,3250462
11,3383452,3,3250462
13,3472425,4,3250462
12,3472422,4,3250462
14,3565164,5,3250462


In [61]:
remain_users = []
for args in  arg_list[:5]:
    remain_users += args
# arg_list = get_parallel_arg_list(n_jobs=12, arg_list=remain_users)
# arg_df_list = []
# for user_ids in arg_list:
tmp_arg = user_df[user_df[uid].isin(remain_users)]
#     print(tmp.shape)
#     arg_df_list.append(tmp)

In [72]:
#========================================================================
# Incremental
#========================================================================

df_base = tmp_arg.copy()
group_no_list = remain_users.copy()

list_user_map = []
for group_no in tqdm(group_no_list):

    group = df_base[df_base[uid]==group_no]
    group.sort_values(by='datetime', inplace=True)
    users = group[col_no].unique().tolist()
    user_map = {}
    cnt = 0

    while True:
        cnt += 1
        if cnt%10==0:
            print(f"loop count: {cnt}")

        remain_user = group[~group[col_no].isin(list(user_map.keys()))][col_no].values
        print(f"* Remain User: {len(remain_user)}")
        if len(remain_user)==0:
            break
    #     remain_user = list( set(users) - set(list(user_map.keys())) )

        base_user = remain_user[0]
        user_map[base_user] = base_user

        base = group[group[col_no]==base_user]
        base_reg = base['Regist_date'].values[0]
        base_dt = base['datetime'].values[0]
        base_d1 = base['D1'].values[0]
        base_d3 = base['D3'].values[0]
        base_c1 = base['C1'].values[0]
        base_c13 = base['C13'].values[0]
        base_v221 = base['V221'].values[0]
        base_v285 = base['V285'].values[0]
        base_hour = base['hour'].values[0]
        base_fraud = base[COLUMN_TARGET].values[0]
        base_is_train = base['is_train'].values[0]

        for i in range(1, len(remain_user), 1):

            tmp_user = remain_user[i]

            tmp = group[group[col_no]==tmp_user]
            tmp_reg = tmp['Regist_date'].values[0]
            diff_regist_days = abs((pd.to_datetime(tmp_reg) - pd.to_datetime(base_reg)).days)

            # Regist_dateで1day以上の差は除外
            if diff_regist_days>1:
                continue

            tmp_dt = tmp['datetime'].values[0]
            tmp_d1 = tmp['D1'].values[0]
            tmp_d3 = tmp['D3'].values[0]
            tmp_c1 = tmp['C1'].values[0]
            tmp_c13 = tmp['C13'].values[0]
            tmp_v221 = tmp['V221'].values[0]
            tmp_v285 = tmp['V285'].values[0]
            tmp_hour = tmp['hour'].values[0]
            tmp_fraud = tmp[COLUMN_TARGET].values[0]
            tmp_is_train = tmp['is_train'].values[0]

            is_same_data = base_is_train == tmp_is_train

#             print("  * Check Data Type Train or Test")
            if not is_same_data:
                if tmp_d3==-99999:
                    match_d1 = False
                else:
                    no_term_date = date_add_days(pd.to_datetime(base_dt), tmp_d3*-1)
                    if train_max_datetime <= no_term_date <= test_min_datetime:
                        match_d1 = True
                    else:
                        match_d1 = False
            else:
                match_d1 = False

            diff_d1   = tmp_d1 - base_d1
            diff_c1   = abs(tmp_c1 - base_c1)
            minus_c1   = tmp_c1 < base_c1
            if minus_c1:
                thres_c1 = 3
                thres_c1_2 = 3
            else:
                thres_c1 = 3
                thres_c1_2 = 6
            thres_c1_large = 5
            thres_v = 3

            diff_c13  = abs(tmp_c13 - base_c13)
            diff_v221 = abs(tmp_v221 - base_v221)
            diff_v285 = abs(tmp_v285 - base_v285)

            ratio_c1   = abs(tmp_c1  / base_c1)
            ratio_c13  = abs(tmp_c13 / base_c13)

    #         print("="*12)
    #         print(f"base {base_user} {base_dt}  ->  D1:{base_d1} D3:{base_d3} C1:{base_c1} C13:{base_c13} V221:{base_v221} V285:{base_v285}")
    #         print(f"tmp  {tmp_user} {tmp_dt}  ->  D1:{tmp_d1} D3:{tmp_d3} C1:{tmp_c1} C13:{tmp_c13} V221:{tmp_v221} V285:{tmp_v285}")
    #         print(f"diff_D1 {diff_d1} diffD1-D3: {diff_d1-tmp_d3} diff_C1:{diff_c1} diff_C13:{diff_c13} diff_V221:{diff_v221} diff_V285:{diff_v285}")
    #         print("="*12)
    
#             print("  * Start User Matching Check.")

            # D3がNullの場合
            if base_d3==-99999 or tmp_d3==-99999:
#                 print("  * D3 Null Pattern")
                # TODO
                # ここでFraud条件を入れると、train-test間を同じユーザーにまとめることはできなくなる
                # 両パターンを作成し、Fraud含めないパターンでまたいたユーザーが本当にまたいでるかチェックしたい

                if diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
                    pass
                elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
                    pass
                elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
                    pass
                elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
                    pass
                elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
                    pass
                elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
                    pass
                elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
                    pass
                elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
                    pass
                else:
                    continue

    #             if base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285<=thres_v:
    #                 pass
    #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221<=thres_v and diff_v285>90000:
    #                 pass
    #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285<=thres_v:
    #                 pass
    #             elif base_fraud==tmp_fraud and diff_c1<=thres_c1 and diff_c13<=thres_c1 and diff_v221>90000 and diff_v285>90000:
    #                 pass
    #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285<=thres_v:
    #                 pa
    #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285>90000:
    #                 pass
    #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221>90000 and diff_v285<=thres_v:
    #                 pass
    #             elif base_fraud==tmp_fraud and base_c1>=20 and tmp_c1>=20 and (0.9<=ratio_c1 and ratio_c1<=1.6) and (0.9<=ratio_c13 or ratio_c13<=1.6) and diff_v221<=thres_v and diff_v285>90000:
    #                 pass
    #             else:
    #                 continue

            # D3がNullsでない場合
            # D3の差分が時差を考慮してもおかしい場合は別ユーザーにする（時差が1~2時間以上だと強制的にはじいてしまうが）
            elif (diff_d1 == tmp_d3) or (abs(diff_d1 - tmp_d3)==1):
                pass
            else:
                # train testの境目
                if not is_same_data and match_d1:
                    pass
                else:
                    # 別ユーザーとみなす
                    continue

            # D3がNullsでない場合
            if base_d3!=-99999 and tmp_d3!=-99999:
#                 print(f"  * D3: {base_d3} -> {tmp_d3}")

                # 別ユーザーとみなす
                if diff_c1<-10 or diff_c1>1000:
                    continue
                if diff_c13<-10 or diff_c13>1000:
                    continue

    #             print("  * check diff_V")
                # V221, 285の変化量が1トランザクションで10以上はないという想定
                if (10<= diff_v221) and (diff_v221<=100):
                    continue
                if (10<= diff_v285) and (diff_v285<=100):
                    continue

#                 print("  * All Check")
                
    #             print("  * check diff_C")
                if diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1>0:
    #                 print('    * pass1')
                    pass
                elif diff_c1<=thres_c1 and diff_c13<=thres_c1 and (diff_v221<=thres_v or diff_v285<=thres_v) and diff_d1==0:
                    pass
                elif base_c1>=thres_c1_large and tmp_c1>=thres_c1_large and (0.4<=ratio_c1 and ratio_c1<=1.6) and (0.4<=ratio_c13 or ratio_c13<=1.6) and diff_d1!=0:
                    pass
                else:
                    # 別ユーザーとみなす
                    continue

            user_map[tmp_user] = base_user
    #         print(pd.Series(user_map))

            # base_userと同じと判定されたので、baseを直近のトランザクションに書き換える
            base_dt    = tmp['datetime'].values[0]
            base_d1    = tmp['D1'].values[0]
            base_d3    = tmp['D3'].values[0]
            base_c1    = tmp['C1'].values[0]
            base_c13   = tmp['C13'].values[0]
            base_v221  = tmp['V221'].values[0]
            base_v285  = tmp['V285'].values[0]
            base_hour  = tmp['hour'].values[0]
            base_fraud = tmp[COLUMN_TARGET].values[0]

        if len(user_map.keys()) == len(users):
            break
#             break

    ser_user = pd.Series(user_map).to_frame('same_user')
    ser_user['group_no'] = group_no
    list_user_map.append(ser_user)





  0%|          | 0/230 [00:00<?, ?it/s]



  0%|          | 1/230 [00:00<00:45,  5.09it/s]



  1%|          | 2/230 [00:00<00:43,  5.30it/s]

* Remain User: 3
* Remain User: 2
* Remain User: 1
* Remain User: 4
* Remain User: 2






  1%|▏         | 3/230 [00:00<00:52,  4.35it/s]

* Remain User: 86
* Remain User: 2
* Remain User: 1






  2%|▏         | 4/230 [00:00<00:52,  4.34it/s]

...

* Remain User: 2
* Remain User: 1
* Remain User: 7225
* Remain User: 7203
* Remain User: 7202
* Remain User: 7200
* Remain User: 7198
* Remain User: 7196
* Remain User: 7177
* Remain User: 7176
* Remain User: 7175
loop count: 10
* Remain User: 7174
* Remain User: 7173
* Remain User: 7165
* Remain User: 7164
* Remain User: 7159
* Remain User: 7158
* Remain User: 7151
* Remain User: 7150
* Remain User: 7148
* Remain User: 7142
loop count: 20
* Remain User: 7141
* Remain User: 7137
* Remain User: 7132
* Remain User: 7113
* Remain User: 7112
* Remain User: 7111
* Remain User: 7110
* Remain User: 7106
* Remain User: 7105
* Remain User: 7104
loop count: 30
* Remain User: 7103
* Remain User: 7089
* Remain User: 7088
* Remain User: 7087
* Remain User: 7082
* Remain User: 7081
* Remain User: 7078
* Remain User: 7058
* Remain User: 7057
* Remain User: 7056
loop count: 40
* Remain User: 7055
* Remain User: 7054
* Remain User: 7053
* Remain User: 7052
* Remain User: 7051
* Remain User: 7050
* Rema





 99%|█████████▉| 228/230 [3:24:29<50:57, 1528.76s/it]

* Remain User: 7
* Remain User: 6
* Remain User: 5
* Remain User: 4
* Remain User: 3
* Remain User: 2
* Remain User: 1






100%|█████████▉| 229/230 [3:24:29<17:50, 1070.21s/it]



100%|██████████| 230/230 [3:24:29<00:00, 749.20s/it] 





* Remain User: 3
* Remain User: 2
* Remain User: 1
* Remain User: 2


In [ ]:
df_arg6 = pd.concat(list_user_map, axis=0)
df_same = pd.concat([df_arg1, df_arg2, df_arg3, df_arg4, df_arg5, df_arg6], axis=0)
df_same

In [127]:
user_df.set_index('no', inplace=True)
user_df['same_user'] = df_same['same_user']
same_user_ids = user_df['same_user'].unique()
same_map = {}
for i, same_id in tqdm(enumerate(same_user_ids[1:])):
    same_map[same_id] = i
cnt = max(same_map.values())+1
new_id_list = []
for same_id in user_df['same_user'].values:
    if same_id in same_map:
        new_id = same_map[same_id]
    else:
        new_id = cnt
        cnt += 1
    new_id_list.append(new_id)
user_df['predicted_user_id'] = new_id_list
user_df.rename(columns={uid:'section_id'}, inplace=True)
user_df.sort_values(by='predicted_user_id', inplace=True)
user_df[[COLUMN_ID, 'predicted_user_id', 'section_id']].to_csv('../output/same_user_pattern/0901__same_user_id__card_addr_pemail_M.csv', index=False)

In [130]:
user_df.rename(columns={uid:'section_id'}, inplace=True)
user_df.sort_values(by='predicted_user_id', inplace=True)
user_df[[COLUMN_ID, 'predicted_user_id', 'section_id']].to_csv('../output/same_user_pattern/0901__same_user_id__card_addr_pemail_M.csv', index=False)